## Test optimisation algorithms against results produced by model with known parameters
Using a similar approach to the preceding notebooks ($i_t = R_t\sum_{\tau<t} i_\tau g_{t-\tau}$),
test various optimisation algorithms using model-created outputs
(so that we know the ideal parameters that the algorithm should be approaching).
So that we can start using these various model components in more sophisticated ways,
I'll begin writing the code as properly encapsulated functions
and tracking a few key model quantities.

In [ ]:
from typing import List
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from scipy.optimize import minimize, shgo
import nevergrad as ng

from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import plot_output_fit

In [ ]:
test_process = [1.8, 2.5, 1.6, 0.7]
n_times = 40
model = RenewalModel(100.0, n_times, 10, 5)
gen_mean = 5.5
gen_sd = 1.8
seed = 1.0
test_results = model.func(gen_mean, gen_sd, np.log(test_process), seed).incidence

### Local optimisation algorithm
#### Process values optimised only
Local optimisation algorithm.
It works for this simple case, but quickly breaks down in many other use cases.

In [ ]:
def calib_func(parameters: List[float], targets: dict) -> float:
    incidence = model.func(gen_mean, gen_sd, parameters, seed).incidence
    return sum([(incidence[t] - d) ** 2 for t, d in enumerate(targets)])

param_bounds = [[-1000.0, np.log(10.0)]] * 4
result = minimize(calib_func, [np.log(2.0)] * 4, method='Nelder-Mead', args=(test_results), bounds=param_bounds)

print(f'target: {test_process}')
print(f'result: {np.exp(result.x)}')
inc = model.func(gen_mean, gen_sd, result.x, seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

#### Generation time parameters included
The same local optimisation algorithm, 
but now additionally including generation time parameters in local optimisation.

In [ ]:
def calib_func(parameters: List[float], targets: dict) -> float:
    gen_mean, gen_sd, *parameters = parameters
    incidence = model.func(gen_mean, gen_sd, parameters, seed).incidence
    return sum([(incidence[t] - d) ** 2 for t, d in enumerate(targets)])

param_bounds = [[1.0, 10.0]] + [[1.0, 5.0]] + [[-10000.0, np.log(10.0)]] * 4
result = minimize(calib_func, [5.0, 1.5] + [np.log(2.0)] * 4, method='Nelder-Mead', args=(test_results), bounds=param_bounds)

print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {result.x[0]}, {result.x[1]}, {np.exp(result.x[2:])}')
inc = model.func(result.x[0], result.x[1], result.x[2:], seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

### Global algorithm
Global optimisation with `scipy`'s `shgo` - need to capture arguments through closure due to bug in optimisation function
as per [comment found online](https://stackoverflow.com/questions/72794609/scipy-issue-passing-arguments-to-optimize-shgo-function).

In [ ]:
global_result = shgo(lambda x, d=test_results: calib_func(x, d), param_bounds)

print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {global_result.x[0]}, {global_result.x[1]}, {np.exp(global_result.x[2:])}')
inc = model.func(global_result.x[0], global_result.x[1], global_result.x[2:], seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

### `nevergrad` algorithms
#### `NGOpt` with default settings
`NGOpt` applied without additional user requests.

In [ ]:
def obj_func(parameters):
    return calib_func(parameters, targets=test_results)

optimizer = ng.optimizers.NGOpt(parametrization=6, budget=2000)
ngopt_result = optimizer.minimize(obj_func)

print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {ngopt_result.value[0]}, {ngopt_result.value[1]}, {np.exp(ngopt_result.value[2:])}')
inc = model.func(ngopt_result.value[0], ngopt_result.value[1], ngopt_result.value[2:], seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

#### `NGOpt` with parameterisation
Specify starting points and bounds for algorithm.

In [ ]:
gen_time_mean_param = ng.p.Scalar(init=5.0, lower=0.1, upper=10.0)
gen_time_sd_param = ng.p.Scalar(init=1.0, lower=0.1, upper=4.0)
process_param = ng.p.Array(init=[0.0] * 4, lower=-10.0, upper=10.0)
instrum = ng.p.Instrumentation(gen_time_mean_param, gen_time_sd_param, process_param)

def obj_func(gen_time_mean, gen_time_sd, parameters):
    return calib_func([gen_time_mean, gen_time_sd] + list(parameters), targets=test_results)

optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=2000)
ngopt_result = optimizer.minimize(obj_func).value[0]  # Zero index gets us the args (not kwargs)
print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {ngopt_result[0]}, {ngopt_result[1]}, {np.exp(ngopt_result[2:])}')
inc = model.func(ngopt_result[0], ngopt_result[1], ngopt_result[2:][0], seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

#### `TwoPointsDE`
An alternative `nevergrad` optimiser.

In [ ]:
optimizer = ng.optimizers.TwoPointsDE(parametrization=instrum, budget=2000)
ngopt_result = optimizer.minimize(obj_func).value[0]

print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {ngopt_result[0]}, {ngopt_result[1]}, {np.exp(ngopt_result[2:])}')
inc = model.func(ngopt_result[0], ngopt_result[1], ngopt_result[2:][0], seed).incidence
pd.DataFrame({'opt': inc, 'target': test_results}).plot()

#### `NGOpt` optimisation with jittered synthetic data
Same as previous best algorithm, but with jitter applied to data.

In [ ]:
spread = 0.2
jitter = pd.Series(np.random.normal(scale=spread, size=n_times) + 1.0)
jittered_vals = test_results * jitter

optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=2000)
jitter_result = optimizer.minimize(obj_func).value[0]

print(f'target: {gen_mean}, {gen_sd}, {test_process}')
print(f'result: {ngopt_result[0]}, {ngopt_result[1]}, {np.exp(ngopt_result[2:])}')
inc = model.func(ngopt_result[0], ngopt_result[1], ngopt_result[2:][0], seed).incidence
pd.DataFrame({'opt': inc, 'target': jittered_vals}).plot()